In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [2]:
import pandas as pd

data = pd.read_csv("spam.csv")


In [3]:
data.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
data.describe()

,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


In [5]:
data.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [6]:
data['spam']=data['Category'].apply(lambda x: 1 if x=='spam' else 0)


In [7]:
data.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['Message'],data['spam'], stratify=data['spam'])

In [9]:
X_train.shape


(4179,)

In [10]:
X_test.shape

(1393,)

In [11]:
y_train.value_counts()

0    3619
1     560
Name: spam, dtype: int64

In [12]:
y_test.value_counts()

0    1206
1     187
Name: spam, dtype: int64

In [13]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [14]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']


In [15]:
get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8435169 , -0.5132728 , -0.8884575 , ..., -0.7474888 ,
        -0.75314736,  0.91964495],
       [-0.8720836 , -0.5054398 , -0.94446677, ..., -0.85847527,
        -0.71745366,  0.8808298 ]], dtype=float32)>

# FUNCTIONAL  MODEL


In [16]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)


In [17]:
# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)


In [18]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [20]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train, epochs=2)

Epoch 1/2
131/131 [==============================] - 1382s 10s/step - loss: 0.3207 - accuracy: 0.8686
Epoch 2/2
131/131 [==============================] - 1056s 8s/step - loss: 0.2287 - accuracy: 0.8997


In [22]:
model.evaluate(X_test, y_test)



44/44 [==============================] - 374s 8s/step - loss: 0.2211 - accuracy: 0.8873


[0.22113792598247528, 0.8872936367988586]

In [23]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [24]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([0, 0, 0, ..., 0, 0, 0])

In [25]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
cm 

array([[1198,    8],
       [ 149,   38]], dtype=int64)

In [26]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.89      0.99      0.94      1206
           1       0.83      0.20      0.33       187

    accuracy                           0.89      1393
   macro avg       0.86      0.60      0.63      1393
weighted avg       0.88      0.89      0.86      1393



In [27]:
reviews = [
    'Enter a chance to win $5000, hurry up, offer valid until march 31, 2021',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
model.predict(reviews)

array([[0.23819214],
       [0.2650075 ],
       [0.27580285],
       [0.03873554],
       [0.02412638]], dtype=float32)